# Langchain with AzureML


## Set up Environment Variables

In [1]:
from dotenv import load_dotenv
load_dotenv()
import openai, os
openai.api_type = "azure"
openai.api_version = "2022-12-01"
openai.api_base = os.environ["OPENAI_API_BASE"]
openai.api_key = os.environ["OPENAI_API_KEY"]

print("OpenAI Endpoint:", openai.api_base)
print("MLFlow Tracking URI: ", os.getenv("MLFLOW_TRACKING_URI"))

OpenAI Endpoint: https://aoai.openai.azure.com/
MLFlow Tracking URI:  azureml://eastus2.api.azureml.ms/mlflow/v1.0/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/ray/providers/Microsoft.MachineLearningServices/workspaces/ray


## Chat Model

In [22]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

chat = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    temperature=0,
    openai_api_version="2023-03-15-preview",
)

reply = chat([SystemMessage(content="You are a friendly chat agent."),
              HumanMessage(content="Hello, how are you?")])

print(reply.content)

Hello! I am a virtual assistant and don't have emotions like humans but I am always ready to help you. How can I assist you today?


## Completion Model

### Zero Shot

In [7]:
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003", temperature=0)
text="""
Classify the following news headline into 1 of the following categories: Business, Tech, Politics, Sport, Entertainment

Headline: Major Retailer Announces Plans to Close Over 100 Stores
Category:
"""

print(llm(text))

Business


### Few Shot

In [6]:
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003", temperature=0)

text="""
Classify the following news headline into 1 of the following categories: Business, Tech, Politics, Sport, Entertainment

Headline 1: Donna Steffensen Is Cooking Up a New Kind of Perfection. The Internet’s most beloved cooking guru has a buzzy new book and a fresh new perspective
Category: Entertainment

Headline 2: Major Retailer Announces Plans to Close Over 100 Stores
Category:
"""

print(llm(text))

Business


### Chain of Reasoning

In [18]:
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003", temperature=0)

text="""
what is the anwer to this equation: 3x - 7 = -2x + 5
"""

print(llm(text))


The answer is x = 4.


In [17]:
from sympy import *
x = symbols('x')
solve(3*x - 7 + 2*x - 5, x)


[12/5]

In [16]:
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003", temperature=0)

text="""
solve the following algebraic equation step-by-step and explain you reasoning at each step: 3x - 7 = -2x + 5
"""

print(llm(text))


Step 1: Add 7 to both sides of the equation.
3x - 7 + 7 = -2x + 5 + 7

Step 2: Simplify the left side of the equation.
3x = -2x + 12

Step 3: Subtract -2x from both sides of the equation.
3x - (-2x) = -2x + 12 - (-2x)

Step 4: Simplify the left side of the equation.
5x = 12

Step 5: Divide both sides of the equation by 5.
x = 12/5

Step 6: Simplify the right side of the equation.
x = 2.4


# Chains

https://python.langchain.com/en/latest/modules/chains/getting_started.html

In [31]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import AzureChatOpenAI

chat = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    temperature=0,
    openai_api_version="2023-03-15-preview",
)

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

from langchain.chains import LLMChain
chain = LLMChain(llm=chat, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("organic snacks"))

json_call: {'module': 'langchain.chat_models.openai', 'name': '_generate', 'args': (AzureChatOpenAI(verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fa0217edfc0>, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key='', request_timeout=60, max_retries=6, streaming=False, n=1, max_tokens=None, deployment_name='gpt-35-turbo', openai_api_type='azure', openai_api_base='', openai_api_version='2023-03-15-preview'), [HumanMessage(content='What is a good name for a company that makes organic snacks?', additional_kwargs={})]), 'kwargs': {'stop': None}}
json_result: generations=[ChatGeneration(text='Here are 5 potential names: \n\n1. EarthBites \n2. Purely Nosh \n3. Wholesome Munchies \n4. FreshFusion \n5. NatureNibbles', generation_info=None, message=AIMessage(content='Here are 5 potential names: \n\n1. EarthBites \n2. Purely Nosh \n3. Wholesome Munchies \n

# Patch Langchain

The first thing I did when I started to work with Langchain was to patch it to get more logging information. Langchain does a great job at abstracting away what is really going on under the hood, in particular when working with more complex chains or agents. However, this abstraction comes at the cost of not being able to see what is really going on.

Instead of patching the code, I could have also use a callback, but I found that monkey patching was easier to use since I didn't have to change the code I was running -- for instance, I could just run an off-the-shelf agent and get the logging information.

What my patch does is to add wrap some key functions that langchain uses and log the parameters that go into them and the value that is returned by them to MLFlow as an artifact.

In [5]:
from src.langchain.patch import patch_langchain

patch_langchain()

# Agent

In [43]:
import mlflow
mlflow.end_run()

In [44]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI, AzureOpenAI
from langchain.schema import HumanMessage
import openai, os

llm = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003", temperature=0)

tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
agent.run("I have 1000 dollars. given today's governement bond rate, how much money would I have in 2 years if I invested them in 2 year government bonds?")




> Entering new AgentExecutor chain...
json_call: {'module': 'langchain.llms.openai', 'name': '_generate', 'args': (AzureOpenAI(cache=None, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fa0217edfc0>, client=<class 'openai.api_resources.completion.Completion'>, model_name='text-davinci-003', temperature=0.0, max_tokens=256, top_p=1, frequency_penalty=0, presence_penalty=0, n=1, best_of=1, model_kwargs={}, openai_api_key=None, batch_size=20, request_timeout=None, logit_bias={}, max_retries=6, streaming=False, deployment_name='text-davinci-003'), ["Answer the following questions as best you can. You have access to the following tools:\n\nSearch: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator: Useful for when you need to answer questions about math.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think ab

'In two years, I will have 1083.26 dollars if I invest 1000 dollars in 2 year government bonds.'

# "Retrieval Augmented Generation"-Style Chain

The main task used here is a simple question answering task on AzureML. The user asks a question and the system answers it. In order to answer the question, the system needs to retrieve some context from a cognitive services search index which is then used to generate the answer. 

Langchain has the `RetrievaQA` chain to easily build a system that does this -- all I need is Retriever that implements the `get_relevant_documents` method that returns a list of documents relevant to the user's query. 

In [24]:
from langchain.docstore.document import Document
from langchain.schema import BaseRetriever, Document
from typing import List
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

class CognitiveSearchRetriever(BaseRetriever):
    def __init__(self, endpoint: str, index_name: str, searchkey: str, top: int = 3):
        self.endpoint = endpoint
        self.index_name = index_name
        self.searchkey = searchkey
        self.top = top
        self.client = SearchClient(endpoint=endpoint, index_name=index_name, credential=AzureKeyCredential(searchkey))

    def get_relevant_documents(self, query: str) -> List[Document]:
        docs = []
        for i in self.client.search(query, top=self.top):
            docs.append(Document(page_content=i['content'], metadata={"sourcefile": i['sourcefile']}))
        return docs

    async def aget_relevant_documents(self, query: str) -> List[Document]:
        pass

I can use the above class to query the search index and generate the answer.

In [25]:
cog_search = CognitiveSearchRetriever(endpoint=os.getenv("COG_SEARCH_ENDPOINT"), index_name="amldocs", searchkey=os.environ["COG_SEARCH_KEY"])
cog_search.get_relevant_documents("pipelines")

[Document(page_content='   > You can publish multiple pipelines to a single endpoint. Each pipeline in a given endpoint is given a version number, which you can specify when you call the pipeline endpoint.\n\n1. Select **Publish**.\n\n## Retrain your model\n\nNow that you have a published training pipeline, you can use it to retrain your model on new data. You can submit jobs from a pipeline endpoint from the studio workspace or programmatically.\n\n### Submit jobs by using the studio portal\n\nUse the following steps to submit a parameterized pipeline endpoint job from the studio portal:\n\n1. Go to the **Endpoints** page in your studio workspace.\n1. Select the **Pipeline endpoints** tab. Then, select your pipeline endpoint.\n1. Select the **Published pipelines** tab. Then, select the pipeline version that you want to run.\n1. Select **Submit**.\n1. In the setup dialog box, you can specify the parameters values for the job. For this example, update the data path to train your model u

Then it takes only a few lines to build a chain that uses that search index to answer the user's question.

In [27]:
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    temperature=0,
    openai_api_version="2023-03-15-preview",
)

qa = RetrievalQA.from_chain_type(llm=llm, 
                                    chain_type="stuff",
                                    retriever=cog_search)

In [28]:
qa("What is a pipeline?")

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


{'query': 'What is a pipeline?',
 'result': 'In the context of this article, a pipeline is a sequence of steps to complete a task in a systematic manner. In Azure Machine Learning context, a pipeline refers to machine learning pipelines which stitch together all the steps in your model training process, such as data preparation, feature extraction, hyperparameter tuning and model evaluation. Machine Learning pipelines can also encompass more than just the training of models.'}

See here for the full source code of the chain: [rag_with_cog_search.py](src/langchain/rag_with_cog_search.py)

Here is a link to a notebook that shows how to use the chain: [tender_carnival_zjr2f2kw](https://ml.azure.com/experiments/id/3b32df62-b496-4f39-8498-2f08f7258493/runs/acebb051-af91-408e-8f7f-edfec7da6101?wsid=/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/ray/workspaces/ray&tid=72f988bf-86f1-41af-91ab-2d7cd011db47#overview)

In [31]:
!python src/langchain/rag_with_cog_search.py --question "how can I set up a compute cluster?" --top 3 

running offline, checking secrets
json_call: {'module': '__main__', 'name': 'get_relevant_documents', 'args': (<__main__.CognitiveSearchRetriever object at 0x7fe869ae4880>, 'how can I set up a compute cluster?'), 'kwargs': {}}
cog_search_top: 3
cog_search_query: how can I set up a compute cluster?
cog_search_docs: [Document(page_content='\n# Create an Azure Machine Learning compute cluster\n\n[!INCLUDE [dev v2](../../includes/machine-learning-dev-v2.md)]\n\n> [!div class="op_single_selector" title1="Select the Azure Machine Learning CLI or SDK version you are using:"]\n> * [v1](v1/how-to-create-attach-compute-cluster.md)\n> * [v2 (current version)](how-to-create-attach-compute-cluster.md)\n\nLearn how to create and manage a [compute cluster](concept-compute-target.md#azure-machine-learning-compute-managed) in your Azure Machine Learning workspace.\n\nYou can use Azure Machine Learning compute cluster to distribute a training or batch inference process across a cluster of CPU or GPU com

## Groundedness

The model might sometimes fill in information that was not provided in the search context or that would be common knowledge. If that happens, we call the reply as not grounded. 

For instance the query below suggests a command `az ml workspace compute show-quota` which doesn't exist in the documentation or the product. Instead `az ml compute list-usage` would be the right command.

In [34]:
!python src/langchain/rag_with_cog_search.py --question "how can I see the quota assigned to my workspace in the v2 cli?"

running offline, checking secrets
json_call: {'module': '__main__', 'name': 'get_relevant_documents', 'args': (<__main__.CognitiveSearchRetriever object at 0x7fb722bb0940>, 'how can I see the quota assigned to my workspace in the v2 cli?'), 'kwargs': {}}
cog_search_top: 3
cog_search_query: how can I see the quota assigned to my workspace in the v2 cli?
cog_search_docs: [Document(page_content='\nAfter deployment, this role becomes available in the specified workspace. Now you can add and assign this role in the Azure portal.\n\nFor more information on custom roles, see [Azure custom roles](../role-based-access-control/custom-roles.md). \n\n### Azure Machine Learning operations\n\nFor more information on the operations (actions and not actions) usable with custom roles, see [Resource provider operations](../role-based-access-control/resource-provider-operations.md#microsoftmachinelearningservices). You can also use the following Azure CLI command to list operations:\n\n```azurecli-intera

[Here](data/amldocs/groundedness/conversation_context.json) is the output of the above command -- it contains both the query, the reply and the context based on which the reply was given.

```json
{
  "query": "how can I see the quota assigned to my workspace in the v2 cli?",
  "result": "You can use the following Azure CLI command to view the quotas for your Machine Learning workspace:\n```\naz ml workspace compute show-quota --name <name> --resource-group <resource-group> [--compute-target <compute-target>] [--ids]\n```\n\nReplace `<name>` with the name of the Machine Learning workspace and `<resource-group>` with the resource group that contains the workspace. You can also add `--compute-target` and `--ids` options if you have specific requirements for viewing compute targets or identity information.",
  "context": [
    {
      "page_content": "\nAfter deployment, this role becomes available in the specified workspace. Now you can add and assign this role in the Azure portal.\n\nFor more information on custom roles, see [Azure custom roles](../role-based-access-control/custom-roles.md). \n\n### Azure Machine Learning operations\n\nFor more information on the operations (actions and not actions) usable with custom roles, see [Resource provider operations](../role-based-access-control/resource-provider-operations.md#microsoftmachinelearningservices). You can also use the following Azure CLI command to list operations:\n\n```azurecli-interactive\naz provider operation show –n Microsoft.MachineLearningServices\n```\n\n## List custom roles\n\nIn the Azure CLI, run the following command:\n\n```azurecli-interactive\naz role definition list --subscription <sub-id> --custom-role-only true\n```\n\nTo view the role definition for a specific custom role, use the following Azure CLI command. The `<role-name>` should be in the same format returned by the command above:\n\n```azurecli-interactive\naz role definition list -n <role-name> --subscription <sub-id>\n```\n\n## Update a custom role\n\nIn the Azure CLI, run the following command:\n\n```azurecli-interactive\naz role definition update --role-definition update_def.json --subscription <sub-id>\n```\n\nYou need to have permissions on the entire scope of your new role definition. For example if this new role has a scope across three subscriptions, you need to have permissions on all three subscriptions. \n\n> [!NOTE]\n> Role updates can take 15 minutes to an hour to apply across all role assignments in that scope.\n\n## Use Azure Resource Manager templates for repeatability\n\nIf you anticipate that you'll need to recreate complex role assignments, an Azure Resource Manager template can be a significant help. The [machine-learning-dependencies-role-assignment template](https://github.com/Azure/azure-quickstart-templates/tree/master//quickstarts/microsoft.machinelearningservices/machine-learning-dependencies-role-assignment) shows how role assignments can be specified in source code for reuse. \n\n## Common scenarios\n\nThe following table is a summary of Azure Machine Learning activities and the permissions required to perform them at the least scope. For example, if an activity can be performed with a workspace scope (Column 4), then all higher scope with that permission will also work automatically. Note that for certain activities the permissions differ between V1 and V2 APIs.\n\n> [!IMPORTANT]\n> All paths in this table that start with `/` are **relative paths** to `Microsoft.MachineLearningServices/` :\n\n| Activity | Subscription-level scope | Resource group-level scope | Workspace-level scope |\n| ----- | ----- | ----- | ----- |\n| Create new workspace <sub>1</sub> | Not required | Owner or contributor | N/A (becomes Owner or inherits higher scope role after creation) |\n| Request subscription level Amlcompute quota or set workspace level quota | Owner, or contributor, or custom role </br>allowing `/locations/updateQuotas/action`</br> at subscription scope | Not Authorized | Not Authorized |\n| Create new compute cluster | Not required | Not required | Owner, contributor, or custom role allowing: `/workspaces/computes/write` |\n| Create new compute instance | Not required | Not required | Owner, contributor, or custom role allowing: `/workspaces/computes/write` |\n| Submitting any type of run (V1) | Not required | Not required | Owner, contributor, or custom role allowing: `\"/workspaces/*/read\", \"/workspaces/environments/write\", \"/workspaces/experiments/runs/write\", \"/workspaces/metadata/artifacts/write\", \"/workspaces/metadata/snapshots/write\", \"/workspaces/environments/build/action\", \"/workspaces/experiments/runs/submit/action\", \"/workspaces/environments/readSecrets/action\"` |\n| Submitting any type of run (V2) | Not required | Not required | Owner, contributor, or custom role allowing: `\"/workspaces/*/read\", \"/workspaces/environments/write\", \"/workspaces/jobs/*\", \"/workspaces/metadata/artifacts/write\", \"/workspaces/metadata/codes/*/write\", \"/workspaces/environments/build/action\", \"/workspaces/environments/readSecrets/action\"` |\n",
      "metadata": {
        "sourcefile": "UI/2023-04-06_191207_UTC/simple-4000-100/how-to-assign-roles-108.md"
      }
    },
    {
      "page_content": "\n## Troubleshooting\n\nHere are a few things to be aware of while you use Azure role-based access control (Azure RBAC):\n\n- When you create a resource in Azure, such as a workspace, you're not directly the owner of the resource. Your role is inherited from the highest scope role that you're authorized against in that subscription. As an example if you're a Network Administrator, and have the permissions to create a Machine Learning workspace, you would be assigned the Network Administrator role against that workspace, and not the Owner role.\n\n- To perform quota operations in a workspace, you need subscription level permissions. This means setting either subscription level quota or workspace level quota for your managed compute resources can only happen if you have write permissions at the subscription scope.\n\n- When there are two role assignments to the same Azure Active Directory user with conflicting sections of Actions/NotActions, your operations listed in NotActions from one role might not take effect if they are also listed as Actions in another role. To learn more about how Azure parses role assignments, read [How Azure RBAC determines if a user has access to a resource](../role-based-access-control/overview.md#how-azure-rbac-determines-if-a-user-has-access-to-a-resource)\n\n- To deploy your compute resources inside a VNet, you need to explicitly have permissions for the following actions:\n    - `Microsoft.Network/virtualNetworks/*/read` on the VNet resources.\n    - `Microsoft.Network/virtualNetworks/subnets/join/action` on the subnet resource.\n    \n    For more information on Azure RBAC with networking, see the [Networking built-in roles](../role-based-access-control/built-in-roles.md#networking).\n\n- It can sometimes take up to 1 hour for your new role assignments to take effect over cached permissions across the stack.\n\n## Next steps\n\n- [Enterprise security overview](concept-enterprise-security.md)\n- [Virtual network isolation and privacy overview](how-to-network-security-overview.md)\n- [Tutorial: Train and deploy a model](tutorial-train-deploy-notebook.md)\n- [Resource provider operations](../role-based-access-control/resource-provider-operations.md#microsoftmachinelearningservices)\n",
      "metadata": {
        "sourcefile": "UI/2023-04-06_191207_UTC/simple-4000-100/how-to-assign-roles-573.md"
      }
    },
    {
      "page_content": "\n## Assign managed identity\n\nYou can assign a system- or user-assigned [managed identity](../active-directory/managed-identities-azure-resources/overview.md) to a compute instance, to authenticate against other Azure resources such as storage. Using managed identities for authentication helps improve workspace security and management. For example, you can allow users to access training data only when logged in to a compute instance. Or use a common user-assigned managed identity to permit access to a specific storage account. \n\nYou can create compute instance with managed identity from Azure ML Studio:\n\n1.\tFill out the form to [create a new compute instance](?tabs=azure-studio#create).\n1.\tSelect **Next: Advanced Settings**.\n1.\tEnable **Assign a managed identity**.\n1.  Select **System-assigned** or **User-assigned** under **Identity type**.\n1.  If you selected **User-assigned**, select subscription and name of the identity.\n\nYou can use V2 CLI to create compute instance with assign system-assigned managed identity:\n\n```azurecli\naz ml compute create --name myinstance --identity-type SystemAssigned --type ComputeInstance --resource-group my-resource-group --workspace-name my-workspace\n```\n\nYou can also use V2 CLI with yaml file, for example to create a compute instance with user-assigned managed identity:\n\n```azurecli\nazure ml compute create --file compute.yaml --resource-group my-resource-group --workspace-name my-workspace\n```\n\nThe identity definition is contained in compute.yaml file:\n\n```yaml\nhttps://azuremlschemas.azureedge.net/latest/computeInstance.schema.json\nname: myinstance\ntype: computeinstance\nidentity:\n  type: user_assigned\n  user_assigned_identities: \n    - resource_id: identity_resource_id\n```\n\nOnce the managed identity is created, grant the managed identity at least Storage Blob Data Reader role on the storage account of the datastore, see [Accessing storage services](how-to-identity-based-service-authentication.md?tabs=cli#accessing-storage-services). Then, when you work on the compute instance, the managed identity is used automatically to authenticate against datastores.\n\n> [!NOTE]\n> The name of the created system managed identity will be in the format /workspace-name/computes/compute-instance-name in your Azure Active Directory. \n\nYou can also use the managed identity manually to authenticate against other Azure resources. The following example shows how to use it to get an Azure Resource Manager access token:\n\n```python\nimport requests\n\ndef get_access_token_msi(resource):\n    client_id = os.environ.get(\"DEFAULT_IDENTITY_CLIENT_ID\", None)\n    resp = requests.get(f\"{os.environ['MSI_ENDPOINT']}?resource={resource}&clientid={client_id}&api-version=2017-09-01\", headers={'Secret': os.environ[\"MSI_SECRET\"]})\n    resp.raise_for_status()\n    return resp.json()[\"access_token\"]\n\narm_access_token = get_access_token_msi(\"https://management.azure.com\")\n```\n\nTo use Azure CLI with the managed identity for authentication, specify the identity client ID as the username when logging in: \n```azurecli\naz login --identity --username $DEFAULT_IDENTITY_CLIENT_ID\n```\n\n> [!NOTE]\n> You cannot use ```azcopy``` when trying to use managed identity. ```azcopy login --identity``` will not work.\n\n## Add custom applications such as RStudio or Posit Workbench (preview)\n\n> [!IMPORTANT]\n> Items marked (preview) below are currently in public preview.\n> The preview version is provided without a service level agreement, and it's not recommended for production workloads. Certain features might not be supported or might have constrained capabilities.\n> For more information, see [Supplemental Terms of Use for Microsoft Azure Previews](https://azure.microsoft.com/support/legal/preview-supplemental-terms/).\n\nYou can set up other applications, such as RStudio, or Posit Workbench (formerly RStudio Workbench), when creating a compute instance. Follow these steps in studio to set up a custom application on your compute instance\n\n1.\tFill out the form to [create a new compute instance](?tabs=azure-studio#create)\n",
      "metadata": {
        "sourcefile": "UI/2023-04-06_191207_UTC/simple-4000-100/how-to-create-manage-compute-instance-534.md"
      }
    }
  ]
}
```

We could now set up a labeling project that would show the user the query, the context and the reply and ask them to label the reply as grounded or not grounded. We will need to do that from time to time, but it would be nice to have an automated way to do this -- if fact, GPT-4 does an OK job at this. It is not perfect, but it is good enough to be useful.

See here for the code of a groundedness check: [groundedness.py](src/langchain/groundedness.py) -- as you can see the meta_prompt reads very much like a briefing you would give to a human labeler.

Now, let's run the groundedness check on the above query and see what it says:

In [35]:
!python src/langchain/groundedness.py --conversation_context data/amldocs/groundedness/conversation_context.json

running offline, checking secrets
json_call: {'module': 'langchain.chat_models.openai', 'name': '_generate', 'args': (AzureChatOpenAI(verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7f8141daafb0>, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-4-32k', temperature=0.0, model_kwargs={}, openai_api_key='', request_timeout=60, max_retries=6, streaming=False, n=1, max_tokens=None, deployment_name='gpt-4-32k', openai_api_type='azure', openai_api_base='', openai_api_version='2023-03-15-preview'), [HumanMessage(content='\n# You will be given a question posed by a user (Human) and the response that was given by a generative AI bot (System). \nThis task is to check if factual information in the System\'s reply is grounded.\n\nThe information is presented to you in a JSON format below with the following fields:\nquery: The question posed by the Human.\nresult: The System\'s reply.\ncontext: The context of the Sys

... it does indeed flag the wrong command as an **ungrounded fact** and gives a groundedness rating of 0/10. So, now we can actually measure the groundedness of our `RetrievalQA` chain -- let's run the groundedness check on all the queries in the test set and see how it does. To do this, we are running both [rag_with_cog_search.py](src/langchain/rag_with_cog_search.py) and [groundedness.py](src/langchain/groundedness.py) in batch and we connect the batches in a pipeline -- this is where AzureML comes in handy. 

Here is the run: [jovial_yogurt_56h2qwvj96](https://ml.azure.com/runs/jovial_yogurt_56h2qwvj96?wsid=/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/ray/workspaces/ray)

Once it has completed, each of the questions have been run through the rag to give us a reply and then the reply has been checked for groundedness. The output of the 

![](images/short-pipeline.png)


Each individual groundedness check is a child run of the groundedness check and we can see the average groundedness value given by GPT-4 as a metric of this run. The average groundedness value is 7.9875 -- it is nothing I would take to the bank, but it is a metric we can look at when we make changes to the rag in order to make it better. After all, that's what we want to do -- we want to systematically make the rag better and better until it is good enough to be useful.

## Compare different Prompts

So, let's try to make it better by changing the prompt used in the RAG. Instead of the default prompt that is being used, let's use one that is asking the model to give us a more concise answer. Instead of the default prompt:

#### System_prompt:
```text
Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}
```

#### User_prompt:
```text
{question}
```

We will use the following prompt: [rag_brief.mime](data/amldocs/prompts/rag_brief.mime)

#### System_prompt:
```text
Use the following pieces of context to answer the users question in a brief way. Don't use more than 100 words. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}
```

#### User_prompt:
```text
Please give a short answer to the following question. Remember to be brief and not use more than 100 words.
And, do not make stuff up -- if you don't know the answer, just say so.
{question}
```

We can run those two prompts in parallel and compare the results.

Here is the run [rate_limit_rerun_loyal_brush_8w6sp6rf9q](https://ml.azure.com/experiments/id/191370e3-619a-48c6-b1ca-09f5498e7e11/runs/45d21b27-0f3d-46ec-86f6-194d8cd1b1a3?wsid=/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/ray/workspaces/ray&tid=72f988bf-86f1-41af-91ab-2d7cd011db47#)

![](images/long-pipeline.png)